# Datos de Previsión de Generación Termosolar

Los datos de previsión de generación son proporcionados por la empresa Meteologica.
Estas previsiones son insertadas de manera diaria en la base de datos de Axpo, empresa que colabora con la realización de este TFM.

La obtención de los datos se realiza de la base de datos SQL de Axpo que actua como concentrador de previsiones de las diferentes instalaciones en la cartera.

A través de la siguiente query se obtienen los datos de previsión horaria para la UFI que representa la unidad de generación objeto de este estudio.

Por LPD se utiliza un Alias.

~~~~sql
SELECT a.FESESION,a.HORA,a.CDUNIFIS,isnull(a.ProgSinEst,0) as PROGRAMA       
FROM (SELECT PMD.[FESESION],PMD.[HORA],UF.[CDUNIFIS],
        CASE WHEN UF.FACTURAR_MIC = 'S' THEN 
        (SELECT ISNULL(SUM(f.[ENERGIA]),0)
            FROM #tmp_aire_programa_final_sesion F 
            WHERE F.[FECHA] = PMD.[FESESION] AND F.[HORA] = PMD.[HORA] AND F.[CDUNIFIS] = UF.[CDUNIFIS]) 
            -(SELECT ISNULL(SUM(EST.[ENERGIAMWH]),0) FROM #tmp_aire_est_energia_ufi AS EST 
            WHERE EST.[FESESION] = PMD.[FESESION] AND EST.[HORA] = PMD.[HORA] AND EST.[CDUNIFIS] = UF.[CDUNIFIS]) 
            +(SELECT ISNULL(SUM(MIC.[PREVISION_MIC]),0) FROM #tmp_aire_prevision_ufis_mic AS MIC 
            WHERE MIC.[FECHA] = PMD.[FESESION] AND MIC.[HORA] = PMD.[HORA] AND MIC.[CDUNIFIS] = UF.[CDUNIFIS]) 
            +(SELECT ISNULL(SUM(RT5.[ENERGIAMW]),0) FROM #tmp_aire_liq_rt5_ufi AS RT5 
            WHERE RT5.[FECHA] = PMD.[FESESION] AND RT5.[HORA] = PMD.[HORA] AND RT5.[CDUNIFIS] = UF.[CDUNIFIS]) 
            +(SELECT ISNULL(SUM(TERC.[ENERGIA_MWH]),0) FROM #VA_am_energia_asig_ufi_est AS TERC 
            WHERE TERC.[FECHA] = PMD.[FESESION] AND TERC.[HORA] = PMD.[HORA] AND TERC.[CDUNIFIS] = UF.[CDUNIFIS]) 
        ELSE
           (SELECT ISNULL(SUM(f.[ENERGIA]),0) FROM #tmp_aire_programa_final_sesion F 
           WHERE F.[FECHA] = PMD.[FESESION] AND F.[HORA] = PMD.[HORA] AND F.[CDUNIFIS] = UF.[CDUNIFIS]) 
           -(SELECT ISNULL(SUM(EST.[ENERGIAMWH]),0) FROM #tmp_aire_est_energia_ufi AS EST 
           WHERE EST.[FESESION] = PMD.[FESESION] AND EST.[HORA] = PMD.[HORA] AND EST.[CDUNIFIS] = UF.[CDUNIFIS]) 
           +(SELECT ISNULL(SUM(RT5.[ENERGIAMW]),0) FROM #tmp_aire_liq_rt5_ufi AS RT5 
           WHERE RT5.[FECHA] = PMD.[FESESION] AND RT5.[HORA] = PMD.[HORA] AND RT5.[CDUNIFIS] = UF.[CDUNIFIS]) 
           +(SELECT ISNULL(SUM(TERC.[ENERGIA_MWH]),0) FROM #VA_am_energia_asig_ufi_est AS TERC 
       WHERE TERC.[FECHA] = PMD.[FESESION] AND TERC.[HORA] = PMD.[HORA] AND TERC.[CDUNIFIS] = UF.[CDUNIFIS]) 
       END ProgSinEst,PMD.precio*10 as prepmd
     FROM #tmp_aire_precios_md PMD INNER JOIN #tmp_aire_fechas_uf UF
     ON PMD.FESESION=UF.FESESION AND PMD.HORA=UF.HORA
     WHERE PMD.FESESION between @FECHASESION and @FECHASESION2
     AND UF.CDUNIFIS IN (@UFISICA)) a
ORDER BY a.[CDUNIFIS],a.[FESESION],a.[HORA]

~~~~
El resultado de esta query se exporta en archivo csv separado por comas